In [15]:
import midi_functions
import numpy as np

In [24]:
song = './Bach-1.mid'
features = midi_functions.get_features(path=song)

input_length = 10 #How many past notes we want to give the LSTM to predict the next one
X, Y = midi_functions.get_XY(features,n_notes=input_length)
X = X[:,np.arange(0,4 * input_length,4)]#gets only the notes and discards velocity/time info
Y = Y[:,0]

In [25]:
X.shape

(699, 10)